<a href="https://colab.research.google.com/github/LukasFeldler/SemAI23/blob/main/UE04_LuFe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Course Instructor: Bernd Neumayr, JKU

# UE04- SPARQL Updata and RDF Datasets

Complete the **8 tasks (1 point per task)** in the `3. SPARQL Update` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- Your solution in executable form: your solutions for SemAI.jar will make use of the default grap. In this notebook you have to transform your solutions according to the workaround exemplified in V04_SPARQL_Update.ipynb
- After executing the update request, print a serizalization of the dataset in TriG format.  

**Task 9 (2 points)**  is to develop a nice visualization of RDF datasets using `visualize_graph_pyvis` from UE02 as a starting point. The requirements are as follows:
- Each named graph must be represented as an independent graph. This means, for example, that :Jane in :JanesGraph is a different node than :Jane in :BillsGraph. There are no edges between nodes in different graphs.
- It is not strictly necessary to draw a box around each named graph, as seen in the slides. The different named graphs should simply be visually distinguishable and not overlap.
- If not all nodes within a named graph are connected, make sure in the visualization that the named graph still forms a coherent visual unit in some way.

In [139]:
# Install required packages
!pip install -q rdflib 

In [140]:
# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace


# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

g = rdflib.Graph()
g.parse(format="turtle", data="""
  prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  prefix : <http://example.org/>
""")

ds = rdflib.Dataset()
ds.parse(format="trig", data="@prefix : <http://example.org/> .")

<Graph identifier=N6d76ed48d64e425f8fcc4f231aa7e4ed (<class 'rdflib.graph.Graph'>)>

Task 1

In [141]:
ds.update("""
  INSERT DATA {
    graph :main {
      :G1 :author :Mary.
      :G2 :author :Peter.
    }  
}
""")

print(g.serialize(format="trig"))

Task 2

In [142]:
ds.update("""
INSERT DATA {
  GRAPH :G1 {
    :Mary :knows :Peter, :John, :Mary.
  }
  GRAPH :G2 {
    :Peter :knows :Mary.
    :John :knows :Mary.
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}




Task 3

In [143]:
ds.update("""
  INSERT {
    graph :main {
      ?known :knownBy ?person.
    }
  
}
WHERE {
  GRAPH :G2 {
    ?person :knows ?known.    
  } 
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .

    :Mary :knownBy :John,
            :Peter .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}




Task 4

In [144]:
ds.update("""
DELETE { 
  graph :main {
    ?s :knownBy ?o .
  }
  
}
WHERE { 
  graph :main {
    ?s :knownBy ?o 
  } 
}

""")
print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}




Task 5

In [145]:
ds.update("""
INSERT {
    graph :main {
        ?graph :knowsCount ?count .
    }
          
  }
  WHERE {
    GRAPH ?graph {
      SELECT (count(*) as ?count)
      Where {
        ?x :knows ?y.
      }
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .

    :main :knowsCount 0 .

    _:N6d76ed48d64e425f8fcc4f231aa7e4ed :knowsCount 0 .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}




Task 6

In [146]:
ds.update("""
  INSERT {  
    graph :main {
        :ds :graphCount ?numGraphs .
    }   
}
WHERE { 
  SELECT (COUNT(DISTINCT ?graph) AS ?numGraphs) 
  WHERE {
    GRAPH ?graph { ?s ?p ?o }
    
  }
}

""")
print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .

    :ds :graphCount 3 .

    :main :knowsCount 0 .

    _:N6d76ed48d64e425f8fcc4f231aa7e4ed :knowsCount 0 .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}




Task 7

In [147]:
ds.update("""
DELETE { 
      ?s ?p ?o    
}
INSERT { 
   GRAPH ?s { 
     ?s ?p ?o 
   } 
}
WHERE {
  graph :main {
    ?s ?p ?o.
  }
    
    
    GRAPH ?s {
    ?s ?p ?o.
  }
  
  
  
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .

    :ds :graphCount 3 .

    :main :knowsCount 0 .

    _:N6d76ed48d64e425f8fcc4f231aa7e4ed :knowsCount 0 .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}




Task 8

In [148]:
ds.update("""
 DELETE {
  GRAPH ?g {
    ?g :knowsCount ?count .
  }
}
INSERT {
  GRAPH ?g {
    ?author :knows :Susi.
    ?g :knowsCount ?newCount .
  }
}
WHERE {
  GRAPH ?g {
    ?g :author ?author .
    ?g :knowsCount ?count .    
  }
  BIND((?count + 1) AS ?newCount)
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .

    :ds :graphCount 3 .

    :main :knowsCount 0 .

    _:N6d76ed48d64e425f8fcc4f231aa7e4ed :knowsCount 0 .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}


